In [6]:
# 거주지역,총매출 데이터 프레임
cursor.execute('select 거주지역, sum(구매금액) 총매출 from region group by 거주지역 order by 거주지역')
x = cursor.fetchall()

df = pd.DataFrame(x, columns=['거주지역','총매출'])
df.head()

,거주지역,총매출
0,010,5894259268
1,013,8692205058
2,016,29914485195
3,020,3056336857
4,024,13670457017


In [4]:
# 거주지역,사용횟수 데이터 프레임
cursor.execute('select 거주지역, count(고객번호1) 사용횟수 from region group by 거주지역 order by 거주지역')
x = cursor.fetchall()

df2 = pd.DataFrame(x, columns=['거주지역','사용횟수'])
df2.head()

,거주지역,사용횟수
0,010,185
1,013,220
2,016,819
3,020,76
4,024,339


In [11]:
# 데이터 프레임 합치기
a = pd.merge(df,df2)
a.head()

,거주지역,총매출,사용횟수
0,010,5894259268,185
1,013,8692205058,220
2,016,29914485195,819
3,020,3056336857,76
4,024,13670457017,339


여기까지 하고 보니까 기존고객이 수정이 안되어 있어서 다시하기 시작

In [12]:
import openpyxl
a.to_excel('region_sum.xlsx')

In [373]:
# 고객속성추가 된거 불러옴
import pandas as pd
import cx_Oracle
import csv
import numpy as np
a = pd.read_csv('./data/고객속성추가수정.csv')
a.head()

,고객번호,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,구매횟수.6,구매횟수.7,총구매액,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7
0,1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,73.0,78.0,"8,995,824.0","11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0"
1,2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,126.0,119.0,"13,898,280.0","10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0"
2,3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,139.0,93.0,"297,954.0","564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0"
3,4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,118.0,107.0,"1,288,767.0","3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0"
4,6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,228.0,219.0,"2,131,469.0","3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0"


In [339]:
# 거주지역,총구매액만 따로 빼서 데이터프레임을 t로 만듬
t = a[['거주지역','총구매액']]

In [286]:
# h = t.groupby('거주지역').sum()

In [287]:
# z = t.groupby('거주지역').count()

In [341]:
# 거주지역을 그룹으로 평균을 구해서 f로 만듬
f = t.groupby('거주지역').mean()

In [342]:
# 내려둔 인덱스인 거주지역을 컬럼으로 올림
f['거주지역'] = f.index

In [343]:
# 실수를 정수로 변환하여 출력
f[['총구매액','거주지역']].astype(int).head()

,총구매액,거주지역
거주지역,,
10,3909577,10
13,5115418,13
16,4135776,16
20,4413226,20
24,4046554,24


In [344]:
# 그룹을  정의
def get_category(총구매액):
    cat=''
    if 총구매액 <= 2000000: cat = '하위'
    elif 총구매액 <= 4000000: cat = '보통'
    elif 총구매액 <= 6000000: cat = '상위'
    else:
        cat = '최상위'
    return cat
f['그룹'] = f.총구매액.apply(lambda x:get_category(x))
f[['총구매액','그룹']].head()
f.head()

,총구매액,거주지역,그룹
거주지역,,,
10,"3,909,577.1",10,보통
13,"5,115,418.0",13,상위
16,"4,135,776.7",16,상위
20,"4,413,226.3",20,상위
24,"4,046,554.3",24,상위


In [345]:
# 총구매액이 겹쳐서 지역별매출평균으로 바꿈
f.rename(columns={'총구매액':'지역별매출평균'},inplace=True)

In [381]:
import openpyxl
f.to_excel('region_sum4.xlsx')

In [346]:
# 거주지역 날림
f.drop(['거주지역'],axis=1,inplace=True)

In [347]:
# 원본파일의 거주지역을 인덱스로 내림
z = a.set_index("거주지역")

In [348]:
# 거주지역을 기준으로 원본파일과
a = pd.merge(f,z, how='right',on='거주지역')
a.head()

,지역별매출평균,그룹,고객번호,성별,연령대,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,...,구매횟수.6,구매횟수.7,총구매액,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7
거주지역,,,,,,,,,,,,,,,,,,,,,
60,"5,742,515.3",상위,1,M,60세이상,121.0,117.0,142.0,214.0,172.0,...,73.0,78.0,"8,995,824.0","11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0"
100,"3,623,211.2",보통,2,M,60세이상,184.0,155.0,139.0,132.0,145.0,...,126.0,119.0,"13,898,280.0","10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0"
33,"2,738,469.3",보통,3,M,60세이상,139.0,209.0,246.0,145.0,87.0,...,139.0,93.0,"297,954.0","564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0"
16,"4,135,776.7",상위,4,F,60세이상,93.0,177.0,85.0,87.0,122.0,...,118.0,107.0,"1,288,767.0","3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0"
240,"1,756,836.9",하위,6,F,60세이상,192.0,192.0,178.0,187.0,237.0,...,228.0,219.0,"2,131,469.0","3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0"


In [349]:
# 인덱스에 있는 거주지역을 컬럼으로 하나 올림
a['거주지역'] = a.index
a.head()

,지역별매출평균,그룹,고객번호,성별,연령대,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,...,구매횟수.7,총구매액,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7,거주지역
거주지역,,,,,,,,,,,,,,,,,,,,,
60,"5,742,515.3",상위,1,M,60세이상,121.0,117.0,142.0,214.0,172.0,...,78.0,"8,995,824.0","11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0",60
100,"3,623,211.2",보통,2,M,60세이상,184.0,155.0,139.0,132.0,145.0,...,119.0,"13,898,280.0","10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0",100
33,"2,738,469.3",보통,3,M,60세이상,139.0,209.0,246.0,145.0,87.0,...,93.0,"297,954.0","564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0",33
16,"4,135,776.7",상위,4,F,60세이상,93.0,177.0,85.0,87.0,122.0,...,107.0,"1,288,767.0","3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0",16
240,"1,756,836.9",하위,6,F,60세이상,192.0,192.0,178.0,187.0,237.0,...,219.0,"2,131,469.0","3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0",240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,"3,623,211.2",보통,19333,F,20대,111.0,170.0,90.0,107.0,132.0,...,90.0,"603,348.0","941,484.0","653,268.0","403,911.0","860,360.0","593,631.0","283,814.0","429,998.0",100
310,"1,600,163.5",하위,19334,M,30대,132.0,497.0,495.0,355.0,235.0,...,48.0,"820,744.0","2,787,328.0","2,909,975.0","2,500,765.0","1,719,235.0","822,816.0","546,127.0","384,952.0",310
540,"3,436,480.4",보통,19335,F,30대,95.0,295.0,274.0,256.0,279.0,...,230.0,"922,771.0","2,866,847.0","2,621,277.0","1,811,375.0","2,224,410.0","3,059,050.0","1,749,762.0","1,661,776.0",540


In [350]:
# 고객번호를 인덱스로 내림
a = a.set_index("고객번호")
# a.drop(['고객번호'],axis=1,inplace=True)

In [376]:
# 최종본확인
a.head()

,고객번호,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,구매횟수.6,구매횟수.7,총구매액,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7
0,1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,73.0,78.0,"8,995,824.0","11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0"
1,2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,126.0,119.0,"13,898,280.0","10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0"
2,3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,139.0,93.0,"297,954.0","564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0"
3,4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,118.0,107.0,"1,288,767.0","3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0"
4,6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,228.0,219.0,"2,131,469.0","3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0"


In [362]:
# 칼럼이름 재정의
a = a[['성별','연령대','거주지역','구매횟수','구매횟수.1','구매횟수.2','구매횟수.3','구매횟수.4','구매횟수.5','구매횟수.6','구매횟수.7','총구매액','총구매액.1','총구매액.2','총구매액.3','총구매액.4','총구매액.5','총구매액.6','총구매액.7','지역별매출평균','그룹']]
a.head()

,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,구매횟수.6,...,총구매액,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7,지역별매출평균,그룹
고객번호,,,,,,,,,,,,,,,,,,,,,
1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,73.0,...,"8,995,824.0","11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0","5,742,515.3",상위
2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,126.0,...,"13,898,280.0","10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0","3,623,211.2",보통
3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,139.0,...,"297,954.0","564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0","2,738,469.3",보통
4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,118.0,...,"1,288,767.0","3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0","4,135,776.7",상위
6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,228.0,...,"2,131,469.0","3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0","1,756,836.9",하위


In [363]:
a['고객번호'] = a.index
a

,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,구매횟수.6,...,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7,지역별매출평균,그룹,고객번호
고객번호,,,,,,,,,,,,,,,,,,,,,
1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,73.0,...,"11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0","5,742,515.3",상위,1
2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,126.0,...,"10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0","3,623,211.2",보통,2
3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,139.0,...,"564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0","2,738,469.3",보통,3
4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,118.0,...,"3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0","4,135,776.7",상위,4
6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,228.0,...,"3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0","1,756,836.9",하위,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,F,20대,100,111.0,170.0,90.0,107.0,132.0,81.0,43.0,...,"941,484.0","653,268.0","403,911.0","860,360.0","593,631.0","283,814.0","429,998.0","3,623,211.2",보통,19333
19334,M,30대,310,132.0,497.0,495.0,355.0,235.0,170.0,32.0,...,"2,787,328.0","2,909,975.0","2,500,765.0","1,719,235.0","822,816.0","546,127.0","384,952.0","1,600,163.5",하위,19334
19335,F,30대,540,95.0,295.0,274.0,256.0,279.0,235.0,209.0,...,"2,866,847.0","2,621,277.0","1,811,375.0","2,224,410.0","3,059,050.0","1,749,762.0","1,661,776.0","3,436,480.4",보통,19335


In [365]:
# 라벨인코딩을 위해서 그룹과 고객번호만 따로 뺌
p = a[['고객번호','그룹']]
p

,고객번호,그룹
고객번호,,
1,1,상위
2,2,보통
3,3,보통
4,4,상위
6,6,하위
...,...,...
19333,19333,보통
19334,19334,하위
19335,19335,보통


In [371]:
# csv로 한번 저장
p.to_csv('sample_re.csv')

In [357]:
# 최종 데이터프레임을 고객데이터라는 이름으로 엑셀형식 저장
a.to_excel('고객데이터.xlsx')

In [358]:
# 최정 데이터프레임을 고객속성데이터추가2 라는 이름으르 csv형식 저장
a.to_csv('고객속성데이터추가2.csv')

sql로 만든 데이터를 불러왔는데 기존고객 수정을 안해서 중간에 폐기함

In [31]:
# 거주지역 몇개 있는지 확인
a.거주지역.unique()

array([ 60, 100,  33,  16, 240,  36,  10,  24,  43, 210, 460,  55,  47,
        39, 440,  87,  65,  13,  49, 360,  69, 410, 340, 500,  72, 610,
        45,  75,  82,  30,  79,  27,  52, 270, 540, 310,  20, 570, 300,
        85, 630], dtype=int64)

In [138]:
import pandas as pd
import openpyxl
aa = pd.read_excel('region_sum.xlsx')
aa.head()

,Unnamed: 0,거주지역,총매출,사용횟수
0,0,10.0,5894259268,185
1,1,13.0,8692205058,220
2,2,16.0,29914485195,819
3,3,20.0,3056336857,76
4,4,24.0,13670457017,339


In [139]:
aa.dropna(inplace=True)
# aa.drop(['총매출','Unnamed: 0'],axis=1)

In [140]:
aa.drop(['Unnamed: 0'],axis=1,inplace=True)

In [141]:
a = pd.merge(a,aa, how='left',on='거주지역')

In [142]:
a['지역별평균'] = a['총매출'] / a['사용횟수']
a

,고객번호,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,...,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7,총매출,사용횟수,지역별평균
0,1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,...,"11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0",31069269883,603,"51,524,494.0"
1,2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,...,"10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0",191158319605,5993,"31,896,933.0"
2,3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,...,"564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0",2354836894,96,"24,529,551.0"
3,4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,...,"3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0",29914485195,819,"36,525,622.9"
4,6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,...,"3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0",2667054260,177,"15,068,103.2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,19333,F,20대,100,111.0,170.0,90.0,107.0,132.0,81.0,...,"941,484.0","653,268.0","403,911.0","860,360.0","593,631.0","283,814.0","429,998.0",191158319605,5993,"31,896,933.0"
18862,19334,M,30대,310,132.0,497.0,495.0,355.0,235.0,170.0,...,"2,787,328.0","2,909,975.0","2,500,765.0","1,719,235.0","822,816.0","546,127.0","384,952.0",3201815904,234,"13,682,973.9"
18863,19335,F,30대,540,95.0,295.0,274.0,256.0,279.0,235.0,...,"2,866,847.0","2,621,277.0","1,811,375.0","2,224,410.0","3,059,050.0","1,749,762.0","1,661,776.0",16765203089,568,"29,516,202.6"
18864,19336,F,30대,39,8.0,78.0,80.0,76.0,63.0,33.0,...,"3,421,816.0","3,570,613.0","5,220,264.0","2,929,416.0","3,887,504.0","7,120,188.0","4,088,791.0",5430064102,119,"45,630,790.8"


In [144]:
a.rename(columns={'총매출':'지역별매출','사용횟수':'지역별횟수'},inplace=True)

In [145]:
pd.set_option('display.float_format', '{:,.1f}'.format)

In [146]:
a

,고객번호,성별,연령대,거주지역,구매횟수,구매횟수.1,구매횟수.2,구매횟수.3,구매횟수.4,구매횟수.5,...,총구매액.1,총구매액.2,총구매액.3,총구매액.4,총구매액.5,총구매액.6,총구매액.7,지역별매출,지역별횟수,지역별평균
0,1,M,60세이상,60,121.0,117.0,142.0,214.0,172.0,140.0,...,"11,108,203.0","12,181,088.0","20,694,107.0","9,408,643.0","9,217,176.0","8,792,000.0","4,375,707.0",31069269883,603,"51,524,494.0"
1,2,M,60세이상,100,184.0,155.0,139.0,132.0,145.0,116.0,...,"10,506,625.0","16,767,689.0","6,279,623.0","18,691,374.0","9,983,629.0","15,589,936.0","9,915,866.0",191158319605,5993,"31,896,933.0"
2,3,M,60세이상,33,139.0,209.0,246.0,145.0,87.0,133.0,...,"564,936.0","732,292.0","349,742.0","282,483.0","335,595.0","474,158.0","213,238.0",2354836894,96,"24,529,551.0"
3,4,F,60세이상,16,93.0,177.0,85.0,87.0,122.0,141.0,...,"3,844,763.0","2,750,419.0","1,616,792.0","1,757,018.0","2,058,510.0","2,395,619.0","1,330,359.0",29914485195,819,"36,525,622.9"
4,6,F,60세이상,240,192.0,192.0,178.0,187.0,237.0,160.0,...,"3,209,003.0","3,573,895.0","2,616,896.0","3,660,327.0","2,970,188.0","3,962,173.0","2,647,175.0",2667054260,177,"15,068,103.2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,19333,F,20대,100,111.0,170.0,90.0,107.0,132.0,81.0,...,"941,484.0","653,268.0","403,911.0","860,360.0","593,631.0","283,814.0","429,998.0",191158319605,5993,"31,896,933.0"
18862,19334,M,30대,310,132.0,497.0,495.0,355.0,235.0,170.0,...,"2,787,328.0","2,909,975.0","2,500,765.0","1,719,235.0","822,816.0","546,127.0","384,952.0",3201815904,234,"13,682,973.9"
18863,19335,F,30대,540,95.0,295.0,274.0,256.0,279.0,235.0,...,"2,866,847.0","2,621,277.0","1,811,375.0","2,224,410.0","3,059,050.0","1,749,762.0","1,661,776.0",16765203089,568,"29,516,202.6"
18864,19336,F,30대,39,8.0,78.0,80.0,76.0,63.0,33.0,...,"3,421,816.0","3,570,613.0","5,220,264.0","2,929,416.0","3,887,504.0","7,120,188.0","4,088,791.0",5430064102,119,"45,630,790.8"


In [1]:
season2

NameError: name 'season2' is not defined